In [ ]:
from tensorflow.keras.layers import GlobalAveragePooling2D,BatchNormalization,Dropout
from tensorflow.keras.applications import MobileNet,MobileNetV2,ResNet50V2,MobileNetV3Small
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model
from tensorflow.python.keras.layers.pooling import GlobalAveragePooling1D,GlobalAveragePooling2D,GlobalAveragePooling3D
from tensorflow.python.keras.layers.recurrent import LSTM
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.optimizers import Nadam

frames=16
channels=3
rows=224
columns=224
path_model=''

In [ ]:
def get_cnnbase(name):

  if name=='MobileNet':
    cnn_base = MobileNet(weights='imagenet', include_top=False, input_tensor=Input(shape=(rows,columns,channels)))
  elif name=='MobileNetV2':
    cnn_base = MobileNetV2(weights='imagenet', include_top=False, input_tensor=Input(shape=(rows,columns,channels)))

  cnn_base.summary()
  return cnn_base

# Load Dataset

In [ ]:
import numpy as np
#Hockey dataset
dataX = np.load('/dataset/hoc/datatrain_75.npy')
labels_id = np.load('/dataset/hoc/labeltrain_75.npy')

#Movie dataset
dataX = np.load('/dataset/movie/datatrain_75.npy')
labels_id = np.load('/dataset/movie/labeltrain_75.npy')

#Violent Flow dataset
dataX = np.load('/dataset/violentflow/datatrain_75.npy')
labels_id = np.load('/dataset/violentflow/labeltrain_75.npy')

#integrat three datasets
dataX = np.load('/dataset/mix/mixdataset_train75.npy')
labels_id = np.load('/dataset/mix/mixdataset_label75.npy')

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(dataX, labels_id, test_size=0.25, random_state=42)

# Create Model

In [ ]:
def model(cnn_base):
  x = cnn_base.output
  cnn = Model(cnn_base.input,x)
  video = Input(shape=(frames,rows,columns,channels))

  cnn.trainable = False

  encoded_frames = TimeDistributed(cnn)(video)
  hidden_layer = Dense(1024, activation="relu")(encoded_frames)
  hidden_layer = GlobalAveragePooling3D()(hidden_layer)
  outputs = Dense(2, activation='softmax')(hidden_layer)
  model = Model([video], outputs)
  return model

# Train Model

In [ ]:
def train_modelBase(name,model_name,bz,l_r):
  from tensorflow.keras.optimizers import SGD
  from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau
  import time
  import numpy as np
  name = ['MobileNet','MobileNetV2']
  cnn_base = get_cnnbase(name)

  if model_name == 'model':
    model = model(cnn_base)

  optimizer_new=SGD(lr=l_r, momentum=0.9, nesterov=True)
  model.compile(optimizer=optimizer_new,loss='binary_crossentropy',metrics=['acc'])
  checkpointer = ModelCheckpoint(filepath='/cnnmodel/'+name+'.hdf5', verbose=1, save_best_only=True)
  history_new_cnlst=model.fit(X_train,y_train,batch_size=bz,epochs=500,validation_data=(X_test,y_test),callbacks=[checkpointer])
  end_time = time.time() - start_time
    _, test_acc_cv = model.evaluate(X_test, y_test, verbose=0)
  return test_acc_cv,end_time

In [ ]:
batchsize = [4,8]
learingrate = [0.01,0.001,0.0001,0.00001]

test_acc_cv,end_time = train_modelBase('MobileNet','model',batchsize,learingrate)
test_acc_cv,end_time = train_modelBase('MobileNetV2','model',batchsize,learingrate)